In [23]:
from IPython.display import clear_output
from data_generator import DataGenerator
from medium_generator import MediumGenerator, disk_func, cosine_func
import numpy as np
from ngsolve import * 
from ngsolve.webgui import Draw

dg = DataGenerator(maxh = (0.02, 0.04))

medium = MediumGenerator(cosine_func) # MediumGenerator(disk_func)

background_params = [{"type": 0, "x": 0.0, "y": 0.0, "r": 0.5, "v": 0.0}]
background_permittivity = medium.generate(background_params)



params  = [{"type": 0, "x": 0, "y": 0, "r": 0.5, "v": 1}]

permittivity  = medium.generate(params)


Mesh generation took 4.47419520499534 seconds


In [ ]:
direction_angles = np.linspace(0, 2 * np.pi, 32, endpoint=False)

cauchy_data = dg.generate_cauchy_data(2 * pi, direction_angles, permittivity)

In [24]:
direction_angles = np.linspace(0, 2 * np.pi, 8, endpoint=False)

for i in range(8):

    kx = 2 * pi * cos(direction_angles[i])
    ky = 2 * pi * sin(direction_angles[i])

    psi1 = CF((exp(1j * kx * x) * exp(1j * ky * y)))

    u_scat = dg.solve(kx, ky, permittivity)
    uB_scat0 = dg.solve(kx, ky, background_permittivity)

    r_B = 1.5
    theta_B = np.linspace(0, 2* pi, 361)[0:-1]

    pts = [(r_B * cos(theta_B[i]), r_B * sin(theta_B[i])) for i in range(len(theta_B))]


    u_scat_vals = [u_scat(dg.mesh(*pt)) for pt in pts]
    grad_u_scat_vals = [ (grad(u_scat)(dg.mesh(*pt))[0] * pt[0] + grad(u_scat)(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]


    for j in range(8):
        p_kx = 2 * pi * cos(direction_angles[j])
        p_ky = 2 * pi * sin(direction_angles[j])

        psi2 = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))

        uB_scat = dg.solve(p_kx, p_ky, background_permittivity)


        # dg.save(u_scat, "scatter", "data/scatter.mat")
        grad_uB_scat = grad(uB_scat)


        uB_scat_vals = [uB_scat(dg.mesh(*pt)) for pt in pts]
        grad_uB_scat_vals = [ (grad_uB_scat(dg.mesh(*pt))[0] * pt[0] + grad_uB_scat(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]


        # define domain B as a disk of radius 1.5
        # generate Cauchy data

        psi_test1 = CF((exp(1j * kx * x) * exp(1j * ky * y)))
        grad_psi_test_X1 = psi_test1.Diff(x)
        grad_psi_test_Y1 = psi_test1.Diff(y)

        psi_vals1 = [psi_test1(dg.mesh(*pt)) for pt in pts]
        grad_psi_vals1 = [ (grad_psi_test_X1(dg.mesh(*pt)) * pt[0] + grad_psi_test_Y1(dg.mesh(*pt)) * pt[1]) / r_B  for pt in pts ]



        psi_test2 = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))
        grad_psi_test_X2 = psi_test2.Diff(x)
        grad_psi_test_Y2 = psi_test2.Diff(y)

        psi_vals2 = [psi_test2(dg.mesh(*pt)) for pt in pts]
        grad_psi_vals2 = [ (grad_psi_test_X2(dg.mesh(*pt)) * pt[0] + grad_psi_test_Y2(dg.mesh(*pt)) * pt[1]) / r_B  for pt in pts ]


        # s = sum( [grad_psi_vals[i] * uB_scat_vals[i] - grad_uB_scat_vals[i] * psi_vals[i] for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)

        # print(s)

        # s = sum( [(grad_uB_scat_vals[i] + grad_psi_vals2[i]) * (u_scat_vals[i]+ psi_vals1[i]) - (grad_u_scat_vals[i]+ grad_psi_vals1[i]) * (uB_scat_vals[i] + psi_vals2[i]) for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)




        true_val = Integrate( (permittivity - background_permittivity) * (uB_scat + psi2) *  (u_scat + psi1), dg.mesh)
        cur_val =  Integrate( (permittivity - background_permittivity) * (uB_scat + psi2) *  (uB_scat0 + psi1), dg.mesh)

        print(i,j, true_val - cur_val)


Solving took 4.4001601450145245 seconds
Solving took 4.2820412990113255 seconds
Solving took 4.449174364999635 seconds
0 0
(-0.014520246366240422-0.019914333060444257j)
(-0.014520287658021052-0.019914449939289692j)
(0.007375022031154462-5.589390147822757e-10j)


Solving took 4.471016543015139 seconds
0 1
(-0.014819218919247194-0.0065974001800572285j)
(-0.014819261499246122-0.0065972585976704174j)
(0.016995734018029184-1.3780408279463969e-09j)


Solving took 4.311684875021456 seconds
0 2
(0.014645630419301621+0.03404208897116703j)
(0.014645628204518478+0.03404210572993449j)
(0.06307755493126997-1.3262555006586692e-09j)


Solving took 4.487488417013083 seconds
0 3
(0.11318288486489904+0.08740820511769572j)
(0.11318292243116242+0.08740835741198011j)
(0.1649798508418666-9.926992280529878e-10j)


Solving took 4.328753631009022 seconds
0 4
(0.1852284154658671+0.11345362926315646j)
(0.18522843706918557+0.11345350203241163j)
(0.2335441339436195+1.3702483222710866e-19j)


Solving took 4.8004722

KeyboardInterrupt: 

In [10]:
r_B = 1.5
theta_B = np.linspace(0, 2* pi, 1025)[0:-1]

pts = [(r_B * cos(theta_B[i]), r_B * sin(theta_B[i])) for i in range(len(theta_B))]

uB_scat_vals = [uB_scat(dg.mesh(*pt)) for pt in pts]
grad_uB_scat_vals = [ (grad_uB_scat(dg.mesh(*pt))[0] * pt[0] + grad_uB_scat(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]


u_scat_vals = [u_scat(dg.mesh(*pt)) for pt in pts]
grad_u_scat_vals = [ (grad(u_scat)(dg.mesh(*pt))[0] * pt[0] + grad(u_scat)(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]

# define the test function
p_kx =  kx
p_ky =  ky
psi_test = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))
grad_psi_test_X = psi_test.Diff(x)
grad_psi_test_Y = psi_test.Diff(y)

psi_vals = [psi_test(dg.mesh(*pt)) for pt in pts]
grad_psi_vals = [ (grad_psi_test_X(dg.mesh(*pt)) * pt[0] + grad_psi_test_Y(dg.mesh(*pt)) * pt[1]) / r_B  for pt in pts ]

# s = sum( [grad_psi_vals[i] * uB_scat_vals[i] - grad_uB_scat_vals[i] * psi_vals[i] for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)

# print(s)

s = sum( [(grad_psi_vals[i] + grad_uB_scat_vals[i]) * (u_scat_vals[i]-uB_scat_vals[i]) - (grad_u_scat_vals[i]- grad_uB_scat_vals[i]) * (psi_vals[i]+uB_scat_vals[i]) for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)
print(s)



Integrate( -(permittivity - background_permittivity) * (uB_scat) *  (uB_scat), dg.mesh)

# r_B = 1.5
# theta_B = np.linspace(0, 2* pi, 1025)[0:-1]

# pts = [(r_B * cos(theta_B[i]), r_B * sin(theta_B[i])) for i in range(len(theta_B))]

# uB_scat_vals = [uB_scat(dg.mesh(*pt)) for pt in pts]
# grad_uB_scat_vals = [ (grad_uB_scat(dg.mesh(*pt))[0] * pt[0] + grad_uB_scat(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]


# u_scat_vals = [u_scat(dg.mesh(*pt)) for pt in pts]
# grad_u_scat_vals = [ (grad(u_scat)(dg.mesh(*pt))[0] * pt[0] + grad(u_scat)(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]

# # define the test function
# p_kx =  0 * pi
# p_ky =  8 * pi 
# psi_test = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))
# grad_psi_test_X = psi_test.Diff(x)
# grad_psi_test_Y = psi_test.Diff(y)

# psi_vals = [psi_test(dg.mesh(*pt)) for pt in pts]
# grad_psi_vals = [ (grad_psi_test_X(dg.mesh(*pt)) * pt[0] + grad_psi_test_Y(dg.mesh(*pt)) * pt[1]) / r_B  for pt in pts ]

# s = sum( [grad_psi_vals[i] * uB_scat_vals[i] - grad_uB_scat_vals[i] * psi_vals[i] for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)

# print(s)

# s = sum( [grad_psi_vals[i] * u_scat_vals[i] - grad_u_scat_vals[i] * psi_vals[i] for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)
# print(s)

(-0.00534794670099449-0.016631727305240263j)


(-0.0049966245780328974+0.006142166691137205j)

In [18]:
scene = Draw(u_scat-uB_scat, dg.mesh)
clear_output() # redraw the scene with a new height
scene.Draw(height="3vh")

WebGuiWidget(layout=Layout(height='3vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.240…